# Sistema de Recomendacion

El Dataset se obtuvo desde Kaggle, la Data perteneciente a la informacion de Youtube México
https://www.kaggle.com/datasnaek/youtube-new

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('Data/MXvideos.csv', encoding='ISO-8859-1')

In [3]:
data.head(2)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,SbOwzAl9ZfQ,17.14.11,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,2017-11-13T06:06:22.000Z,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg,False,False,False,Disfruta la presencia del Chef Torreblanca en ...
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,2017-11-13T05:11:58.000Z,La Voz Mexico 7,104972,271,174,369,https://i.ytimg.com/vi/klOV6Xh-DnI/default.jpg,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...


In [4]:
data.shape

(40451, 16)

In [5]:
data.isnull().sum()

video_id                     0
trending_date                0
title                        0
channel_title                0
category_id                  0
publish_time                 0
tags                         0
views                        0
likes                        0
dislikes                     0
comment_count                0
thumbnail_link               0
comments_disabled            0
ratings_disabled             0
video_error_or_removed       0
description               4224
dtype: int64

In [6]:
print('<-- Canales más frecuentes -->\n ')
data.channel_title.value_counts().head(10)

<-- Canales más frecuentes -->
 


Cracks                  186
Badabun                 184
Troom Troom Es          168
Cracks MX               162
Las Estrellas           155
Televisa Telenovelas    149
El Salvador 4K          145
Tu COSMOPOLIS           143
EnamorÃ¡ndonos          136
Ventaneando             133
Name: channel_title, dtype: int64

In [7]:
print('<-- Top 10 Videos con mas dislikes -->\n ')
top_dislikes = data.dislikes.value_counts(ascending=True)
i = 0
while i <= 9:
    ids = top_dislikes.index[i]
    ids = data[data['dislikes'] == ids]
    print('-->', ids.title.iloc[0])
    i = i + 1

<-- Top 10 Videos con mas dislikes -->
 
--> Â¿QuÃ© pasarÃ­a si nunca te baÃ±aras?
--> HACKS PARA UNA NAVIDAD CON CALOR | KAREN POLINESIA MUSAS LOS POLINESIOS
--> ROCKERO Y YOUTUBERO: LAS MADURAS DE MI BARRIO | JORDI ENP
--> âº Tag de la Primera Cita | Â¿Mi nueva novia?
--> Christopher Robin Official Teaser Trailer
--> CHILENA, ERROR, LÃGRIMAS: Â¡MADRlD CAMPE0N! | Â¿CRISTIANO SE DESPIDIÃ del EQUIPO? FLORENTIN0 RESPONDE
--> NiÃ±a bien vota por ya sabes quien.
--> Los 7 himnos nacionales mÃ¡s Ã©picos de la historia
--> HAZ SLIME CON 1 INGREDIENTE ð¦ LICUADORA SLIME Y MÃ¡s Recetas LOCAS!
--> LAS MEJORES BROMAS PESADAS DEL 2018


In [8]:
print('<-- Top 10 Videos con mas likes -->\n ')
top_likes = data.likes.value_counts(ascending=True)
i = 0
while i <= 9:
    ids = top_likes.index[i]
    ids = data[data['likes'] == ids]
    print('-->', ids.title.iloc[0])
    i = i + 1

<-- Top 10 Videos con mas likes -->
 
--> BÃRNS, Lana Del Rey - God Save Our Young Blood (Audio)
--> Luis Miguel La Serie | Teaser: Mi Historia | Netflix
--> Â¿QUÃ OCURRE SI HACES SLIME EN LA LICUADORA? - Experimento
--> TOP 10 ZAPATOS MÃS CAROS DEL MUNDO! | REACCIONANDO
--> PLANETAS COMESTIBLES | MUSAS LESSLIE LOS POLINESIOS | LOLLI POP  | AVENGERS INFINITY WAR
--> Franco Escamilla.- La mesa reÃ±oÃ±a 80 Especial Navidad
--> Karol G, Cosculluela - La Dama
--> 24 FRASES INMORTALES DE ACZINO
--> El âhomenaje de Zidane a Messi | âel mejor jugador del mundo
--> PROMO: Insomnio


In [9]:
#Solo manejo el 40% de los datos
df = data.sample(frac=0.04)
df.reset_index(drop=True, inplace=True)

In [10]:
df.tags.head(4)

0                                               [none]
1                                               [none]
2                                               [none]
3    santa misa del dÃ­a|"eucaristÃ­a del dÃ­a"|"mi...
Name: tags, dtype: object

In [11]:
#Remuevo signos y simbolos presentes en la columna tags
df['xtags'] = df['tags'].str.replace('[{}]'.format(string.punctuation), ' ')

# Convertir palabras a numeros

In [12]:
stopwords = nltk.corpus.stopwords.words('spanish')    #Defino las stopwords
tfidf = TfidfVectorizer(stop_words=set(stopwords))    #Defino el modelo
df['xtags'] =df['xtags'].fillna('')               #Imputar valores
tfidf_matrix = tfidf.fit_transform(df['xtags'])       #Defino la matrix

In [13]:
print(len(tfidf.vocabulary_))
print(tfidf_matrix.shape)

9716
(1618, 9716)


In [14]:
'''
Matriz de comparación
Compara las etiquetas de un video seleccionado con la lista anterior
en busca de palabras similares
'''
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [15]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [16]:
def get_recomendation(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:5]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

# Recomendaciones

In [17]:
#Seleccion de video al azar
video =np.random.randint(df.shape[0], size=1)
video = str(df.iloc[video[0]]['title'])
video

'Noticias irresistibles: Andrea Legarreta vs Magda, Atala Sarmiento y William Levy'

In [18]:
def recomendaciones(video):
    print('Recomendaciones basadas en su ultima reproducción: {} '.format(video))
    print('<------------------------------------------>')
    print(get_recomendation(video))

In [19]:
recomendaciones(video)

Recomendaciones basadas en su ultima reproducción: Noticias irresistibles: Andrea Legarreta vs Magda, Atala Sarmiento y William Levy 
<------------------------------------------>
331    Paty Chapoy se la Canta a Atala: Â¡Si hay Vide...
858    Paul Stanley en Hoy, Mishelle en tanga, Gloria...
877    El Negro Araiza y Andrea Legarreta hacen las p...
180        Se busca video de Atala Sarmiento en Televisa
Name: title, dtype: object
